# TASK 2B

In [6]:
# !pip install datasets
# !pip install transformers
# !pip install Sentencepiece
!pip install nltk


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 660.6 kB/s eta 0:00:03
   -------- ------------------------------- 0.3/1.5 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------  1.5/1.5 MB 12.0 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 9.6 MB/s eta 0:00:00


In [1]:
# download hugging face datasets
import datasets
import sentencepiece
from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments




In [61]:
train_Data = datasets.load_dataset("wmt16","de-en", split="train[:50000]")
val_Data = datasets.load_dataset("wmt16","de-en", split="validation")
test_Data = datasets.load_dataset("wmt16","de-en", split="test")

In [62]:
device = "cuda" if torch.cuda.is_available() else "cpu" 

### Loading the model

In [63]:
model_name = "google-t5/t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
print(val_Data)
# take only 10 samples for testing
# validation_data = []
# for i in range(10):
#     validation_data.append(val_Data[i])
# print(validation_data)

Dataset({
    features: ['translation'],
    num_rows: 2169
})


In [30]:
translations_validation = []
translations_testing = []
validation_data = val_Data
# testing_data = test_Data


# for data in [validation_data, testing_data]:
for example in tqdm(validation_data):
    # Prepend the prefix for translation task
    input_text = "translate English to German: " + example["translation"]["de"]
    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    # Generate translation
    translated_ids = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
    # Decode the translated ids
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    # if data == validation_data:
    translations_validation.append(translated_text)
    # else:
        # translations_testing.append(translated_text)

# Calculate BLEU score for validation and testing translations
reference_validation = [example["translation"]["en"] for example in validation_data]
# reference_testing = [example["translation"]["en"] for example in testing_data]

bleu_score_validation = corpus_bleu([[ref] for ref in reference_validation], translations_validation)
# bleu_score_testing = corpus_bleu([[ref] for ref in reference_testing], translations_testing)

print("BLEU score for validation set:", bleu_score_validation)
# print("BLEU score for testing set:", bleu_score_testing)

100%|██████████| 2169/2169 [38:49<00:00,  1.07s/it] 


BLEU score for validation set: 0.2647026391581188


In [74]:
meteor = datasets.load_metric("meteor")
meteor_score = meteor.compute(predictions=translations_validation, references=reference_validation)
print("METEOR score for validation set:", meteor_score)

# install nltk


c:\Users\Vartika\anaconda3\envs\dl\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for meteor contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/meteor/meteor.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vartika\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vartika\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Vartika\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to

NameError: name 'translations_validation' is not defined

In [ ]:
# use meteor from hugging face evaluate
# meteor = datasets.load_metric("meteor")

In [ ]:
# install bert_score
!pip install bert_score

In [ ]:
# Bert Score
from bert_score import score
P, R, F1 = score(translations_validation, reference_validation, lang="en", verbose=True)
print("BERT Score for validation set:", F1.mean().item())


In [8]:
# take only 10 samples of training data for testing
t_data = []
for i in range(10):
    t_data.append(train_Data[i])

v_Data = []
for i in range(10):
    v_Data.append(val_Data[i])

te_data = []
for i in range(10):
    te_data.append(test_Data[i])


print(t_data)
print(v_Data)
print(te_data)


[{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}}, {'translation': {'de': 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.', 'en': 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'}}, {'translation': {'de': 'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.', 'en': "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful."}}, {'translation': {'de':

# Task 2C

In [64]:
import transformers
from datasets import load_dataset, load_metric
import nltk
nltk.download('punkt')
import string
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vartika\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [79]:

tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

c:\Users\Vartika\anaconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vartika\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [80]:
source_lang = "de" 
target_lang = "en"
prefix = "translate German to English: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [84]:
pt_data = train_Data.map(preprocess_function, batched=True)
pt_data_v = val_Data.map(preprocess_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

c:\Users\Vartika\anaconda3\envs\dl\Lib\site-packages\transformers\tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2169 [00:00<?, ? examples/s]

In [82]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [83]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [88]:
!pip install wandb

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB 8.0 MB/s eta 0:00:01
   ------ --------------------------------- 0.3/2.2 MB 4.3 MB/s eta 0:00:01
   ---------- ----------------------------- 0.6/2.2 MB 4.3 MB/s eta 0:00:01
   ------------ --------------------------- 0.7/2.2 MB 4.0 MB/s eta 0:00:01
   ------------- -------------------------- 0.7/2.2 MB 3.3 MB/s eta 0:00:01
   --------------- ------------------------ 0.8/2.2 MB 3.5 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.2 MB 3.0 MB/s eta 0:00:01
   ------------------- -------------------- 1.1/2.2 MB 3.1 MB/s eta 0:00:01
   -------------------- ------------------- 1.2/2.2 MB 2.9 MB/s eta 0:00:01
   ------------------------ --------------- 1.4/2.2 MB 3.0 MB/s eta 0:00:01
   -------------------------- ------------- 1.5/2.2 MB 3.0 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.2 MB 3.0 MB/s eta 0:00:01
   ----------------

In [95]:

model_config = dict(task = "2C",
                    model_name = "t5-small",
                    learning_rate = 2e-5,
                    per_device_train_batch_size = 16,
                    per_device_eval_batch_size = 16,
                    weight_decay = 0.01,
                    num_train_epochs = 5,
                    save_total_limit = 3,
)

In [96]:
import wandb
wandb.login(relogin=True)
# Initialize wandb
wandb.init(project="assignment-3", entity="nlp-assignments", name="t5-small_task2C", config=model_config)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:




training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    report_to="wandb",
    logging_dir='./logs',
    # fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=pt_data,
    eval_dataset=pt_data_v,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
# save the model
trainer.save_model("fine_tuned_t5_small_task2C")

In [ ]:
## load the saved model
model = AutoModelForSeq2SeqLM.from_pretrained("fine_tuned_t5_small_task2C")
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_t5_small_task2C")

# Preprocess the testing data
t_data = test_Data.map(preprocess_function, batched=True)

# Evaluate the model on the testing data
results = trainer.predict(t_data)
print(results.metrics)

# Calculate BLEU score for validation and testing translations
reference_testing = [example["translation"]["en"] for example in te_data]
translations_testing = results.predictions
bleu_score_testing = corpus_bleu([[ref] for ref in reference_testing], translations_testing)
print("BLEU score for testing set:", bleu_score_testing)

meteor = datasets.load_metric("meteor")
meteor_score = meteor.compute(predictions=translations_testing, references=reference_testing)
print("METEOR score for testing set:", meteor_score)

# Bert Score
from bert_score import score
P, R, F1 = score(translations_testing, reference_testing, lang="en", verbose=True)
print("BERT Score for testing set:", F1.mean().item())
